In [ ]:

import os
import re
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

# ==================== 1. 데이터 로드 함수 (사용자의 기존 코드) ====================

def parse_date_from_filename(filename):
    patterns = [
        r'(\d{4})-(\d{2})-(\d{2})',
        r'(\d{4})(\d{2})(\d{2})',
        r'(\d{2})-(\d{2})-(\d{4})',
        r'(\d{2})(\d{2})(\d{4})'
    ]
    basename = os.path.basename(filename)
    for pattern in patterns:
        match = re.search(pattern, basename)
        if match:
            try:
                if len(match.group(1)) == 4:
                    year, month, day = match.groups()
                else:
                    day, month, year = match.groups()
                return pd.to_datetime(f"{year}-{month}-{day}")
            except:
                continue
    return None

def load_all_news_data(root_dir):
    all_data = []
    if not os.path.exists(root_dir):
        print(f"경고: 디렉토리가 존재하지 않습니다: {root_dir}")
        # 테스트용 더미 데이터 생성
        dates = pd.date_range('2021-01-01', '2024-12-31', freq='D')
        return pd.DataFrame({
            'date': dates,
            'news': ['test news'] * len(dates),
            'label': np.random.choice([1, 0, -1], len(dates))
        })
    
    csv_files = sorted([f for f in os.listdir(root_dir) if f.endswith('.csv')])
    for filename in csv_files:
        filepath = os.path.join(root_dir, filename)
        file_date = parse_date_from_filename(filename)
        for enc in ['utf-8', 'cp949', 'latin1']:
            try:
                df = pd.read_csv(filepath, encoding=enc)
                break
            except UnicodeDecodeError:
                continue
        else:
            print(f"읽기 실패: {filepath}")
            continue
        df = df.loc[:,['news','label'] if 'date' not in df.columns else ['news','label','date']]
        if 'date' not in df.columns:
            df['date'] = file_date
        else:
            df['date'] = pd.to_datetime(df['date'],errors='coerce')
            if file_date is not None:
                df['date'] = df['date'].fillna(file_date)
        all_data.append(df)
    
    if len(all_data) == 0:
        print("경고: CSV 파일을 찾을 수 없습니다. 더미 데이터 생성")
        dates = pd.date_range('2021-01-01', '2024-12-31', freq='D')
        return pd.DataFrame({
            'date': dates,
            'news': ['test news'] * len(dates),
            'label': np.random.choice([1, 0, -1], len(dates))
        })
    
    combined_df = pd.concat(all_data, ignore_index=True)
    combined_df['date'] = pd.to_datetime(combined_df['date'], errors='coerce')
    return combined_df

print("데이터 로드 함수 준비 완료")


In [9]:

# ==================== 2. 실제 데이터 다운로드 (이더리움 가격 데이터) ====================

import yfinance as yf

import pandas as pd
import yfinance as yf
from datetime import datetime

# 데이터 다운로드 기간 설정
START_DATE = '2020-01-01'
END_DATE = '2025-10-04' 


# --- 1. 이더리움 데이터 다운로드 및 정리 ---
eth_price = yf.download('ETH-USD', start=START_DATE, end=END_DATE, progress=False)

# MultiIndex 확인 및 처리: 컬럼이 MultiIndex일 경우 두 번째 레벨 (티커명)을 제거
if isinstance(eth_price.columns, pd.MultiIndex):
    print("⚠️ 이더리움 데이터 컬럼이 MultiIndex입니다. 평탄화 중...")
    # 두 번째 레벨(티커명)을 제거하여 첫 번째 레벨(Open, High, ...)만 남김
    eth_price.columns = eth_price.columns.droplevel(1) 

# 컬럼 이름을 소문자로 변경 (이제 컬럼은 문자열입니다)
eth_price.columns = [col.lower() for col in eth_price.columns]
eth_price.index.name = 'date'
print(f"✓ 이더리움 데이터: {len(eth_price)}일, {eth_price.index.min()} ~ {eth_price.index.max()}")

# --- 2. 상위 5개 암호화폐 데이터 다운로드 (MultiIndex 처리) ---
print("\n상위 5개 암호화폐 데이터 다운로드 중...")
crypto_tickers = ['BTC-USD', 'BNB-USD', 'XRP-USD', 'ADA-USD', 'SOL-USD']
top5_crypto_multi = yf.download(crypto_tickers, start=START_DATE, end=END_DATE, progress=False)

# MultiIndex 컬럼 평탄화 및 이름 정리 (기존 로직 유지)
if isinstance(top5_crypto_multi.columns, pd.MultiIndex):
    print("MultiIndex 컬럼 평탄화 및 이름 정리 중...")
    
    # 튜플의 원소를 '_'로 연결하여 단일 컬럼 이름 생성
    new_cols = []
    for col_tuple in top5_crypto_multi.columns:
        # 예: ('Close', 'BTC-USD') -> 'close_btc'
        col_name_joined = '_'.join(col_tuple) 
        new_name = col_name_joined.lower().replace('-usd', '').replace('adj close', 'adj_close').replace(' ', '_')
        new_cols.append(new_name)
    
    top5_crypto = top5_crypto_multi.copy()
    top5_crypto.columns = new_cols
    top5_crypto.index.name = 'date'

    print(f"✓ 상위 5개 암호화폐 데이터 통합 완료: {top5_crypto.shape}")
    print("\n최종 데이터프레임 컬럼 예시:")
    print(top5_crypto.columns[:10])
else:
    top5_crypto = top5_crypto_multi
    print("MultiIndex가 아닙니다. 추가 정리 불필요.")

# --- 4. 모든 데이터 병합 (선택 사항) ---
# 이더리움 데이터를 포함하여 하나의 DataFrame으로 통합
all_crypto = pd.merge(eth_price, top5_crypto, on='date', how='inner', suffixes=('_eth', None))
all_crypto = all_crypto.rename(columns={'open': 'open_eth', 'high': 'high_eth', 'low': 'low_eth', 'close': 'close_eth', 'volume': 'volume_eth', 'adj close': 'adj_close_eth'})

print(f"\n✓ 모든 암호화폐 데이터 통합 완료: {all_crypto.shape}")
print("\n최종 데이터프레임 (all_crypto) 상위 5개 행:")
print(all_crypto.head())

# 온체인 데이터 로드 시도
print("\n온체인 데이터 로드 시도...")
try:
    onchain = pd.read_csv('eth_onchain.csv', parse_dates=['date'])
    onchain.set_index('date', inplace=True)
    onchain.index = pd.to_datetime(onchain.index)
    print(f"✓ 온체인 데이터: {len(onchain)}일, {onchain.index.min()} ~ {onchain.index.max()}")
    print(f"  컬럼: {list(onchain.columns)}")
except FileNotFoundError:
    print("✗ eth_onchain.csv 파일을 찾을 수 없습니다.")
    print("  온체인 데이터 없이 진행합니다.")
    onchain = None

# 뉴스 감정 데이터 로드
print("\n감정 분석 데이터 로드 시도...")
ROOT_DIR = "./news_data"  # 기본 경로
news_df = load_all_news_data(ROOT_DIR)
print(f"✓ 뉴스 데이터: {len(news_df)}건, {news_df['date'].min()} ~ {news_df['date'].max()}")

# 일별 감정 점수 집계
daily_sentiment = news_df.groupby('date').agg({
    'label': ['mean', 'std', 'count']
})
daily_sentiment.columns = ['sentiment_mean', 'sentiment_std', 'sentiment_count']
daily_sentiment['sentiment_std'] = daily_sentiment['sentiment_std'].fillna(0)
print(f"✓ 일별 감정 데이터: {len(daily_sentiment)}일")

print("\n" + "="*60)
print("모든 데이터 로드 완료!")
print("="*60)


⚠️ 이더리움 데이터 컬럼이 MultiIndex입니다. 평탄화 중...
✓ 이더리움 데이터: 2103일, 2020-01-01 00:00:00 ~ 2025-10-03 00:00:00

상위 5개 암호화폐 데이터 다운로드 중...
MultiIndex 컬럼 평탄화 및 이름 정리 중...
✓ 상위 5개 암호화폐 데이터 통합 완료: (2103, 25)

최종 데이터프레임 컬럼 예시:
Index(['close_ada', 'close_bnb', 'close_btc', 'close_sol', 'close_xrp',
       'high_ada', 'high_bnb', 'high_btc', 'high_sol', 'high_xrp'],
      dtype='object')

✓ 모든 암호화폐 데이터 통합 완료: (2103, 30)

최종 데이터프레임 (all_crypto) 상위 5개 행:
             close_eth    high_eth     low_eth    open_eth   volume_eth  \
date                                                                      
2020-01-01  130.802002  132.835358  129.198288  129.630661   7935230330   
2020-01-02  127.410179  130.820038  126.954910  130.820038   8032709256   
2020-01-03  134.171707  134.554016  126.490021  127.411263  10476845358   
2020-01-04  135.069366  136.052719  133.040558  134.168518   7430904515   
2020-01-05  136.276779  139.410202  135.045624  135.072098   7526675353   

            close_ada  close_bnb   

In [12]:
news_df.head()

,news,label,date
0,"세계 최대 암호화폐인 비트코인(Bitcoin, BTC)은 전일대비 1.54% 하락한...",-1,2020-01-01
1,업비트 암호화폐(가상화폐) 거래소 오전 9시 25분(한국시간) 기준으로 비트코인은 ...,-1,2020-01-01
2,"지난 24시간 동안 세계 최대 암호화폐인 비트코인(Bitcoin, BTC)은 단기 ...",0,2020-01-02
3,업비트 암호화폐(가상화폐) 거래소 오전 9시 50분(한국시간) 기준으로 비트코인은 ...,0,2020-01-02
4,이더리움(Ethereum) 네트워크가 '빙하기'를 늦추기 위한 긴급 하드포크 ‘뮤어...,1,2020-01-03


In [13]:
onchain.head()


,tx_count,active_addresses,new_addresses,large_eth_transfers,token_transfers,contract_events,avg_gas_price,total_gas_used,avg_block_size,avg_block_difficulty
date,,,,,,,,,,
2015-08-07,2050,784,784,283,0,0,6.046842e+11,49353826,632.63,1.470839e+12
2015-08-08,2881,605,430,186,0,6,3.227136e+11,376006093,667.59,1.586124e+12
2015-08-09,1329,462,252,124,0,11,4.754671e+11,38863003,618.30,1.709480e+12
2015-08-10,2037,821,632,115,0,22,4.216549e+11,74070061,631.19,1.837696e+12
2015-08-11,4963,2132,1881,150,0,42,7.783882e+10,163481740,692.01,2.036391e+12


In [10]:

# ==================== 3. CryptoPulse 모델 완전 구현 ====================

class CryptoPulseModel:
    """
    CryptoPulse: Short-Term Cryptocurrency Forecasting with Dual-Prediction
    and Cross-Correlated Market Indicators (2025)
    
    논문의 이중 예측 메커니즘을 정확히 구현
    """
    
    def __init__(self, sequence_length=7):
        self.sequence_length = sequence_length
        self.macro_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.05, 
                                                      max_depth=5, random_state=42)
        self.dynamics_model = Ridge(alpha=1.0)
        self.fusion_model = GradientBoostingRegressor(n_estimators=50, learning_rate=0.1, 
                                                       max_depth=3, random_state=42)
        
        self.scaler_macro = StandardScaler()
        self.scaler_dynamics = StandardScaler()
        self.scaler_fusion = StandardScaler()
        
        self.trained = False
        
    def compute_technical_indicators(self, df):
        """
        논문 Section III-A: 7개 기술적 지표 계산
        """
        result = df.copy()
        
        # Stochastic %K (N=14)
        result['lowest_14'] = result['low'].rolling(14, min_periods=1).min()
        result['highest_14'] = result['high'].rolling(14, min_periods=1).max()
        result['stochastic_k'] = ((result['close'] - result['lowest_14']) / 
                                 (result['highest_14'] - result['lowest_14'] + 1e-10)) * 100
        
        # Stochastic %D (3-day SMA of %K)
        result['stochastic_d'] = result['stochastic_k'].rolling(3, min_periods=1).mean()
        
        # Williams %R (N=14)
        result['williams_r'] = ((result['highest_14'] - result['close']) / 
                               (result['highest_14'] - result['lowest_14'] + 1e-10)) * 100
        
        # A/D Oscillator
        result['ad_oscillator'] = ((result['close'].shift(1) - result['close']) / 
                                  (result['high'] - result['low'] + 1e-10))
        result['ad_oscillator'] = result['ad_oscillator'].fillna(0)
        
        # Momentum (N=10)
        result['momentum'] = result['close'] - result['close'].shift(10)
        result['momentum'] = result['momentum'].fillna(0)
        
        # Disparity 7
        result['ma_7'] = result['close'].rolling(7, min_periods=1).mean()
        result['disparity_7'] = (result['close'] / (result['ma_7'] + 1e-10)) * 100
        
        # Rate of Change (N=12)
        result['roc'] = (result['close'] / (result['close'].shift(12) + 1e-10)) * 100
        result['roc'] = result['roc'].fillna(100)
        
        # 중간 계산 컬럼 제거
        technical_cols = ['open', 'high', 'low', 'close', 'volume',
                         'stochastic_k', 'stochastic_d', 'williams_r', 
                         'ad_oscillator', 'momentum', 'disparity_7', 'roc']
        
        return result[technical_cols]
    
    def create_sequences(self, price_data, macro_data, sentiment_data, onchain_data=None):
        """
        논문의 시퀀스 구조 생성: L=7일 관찰 윈도우
        """
        sequences = []
        
        # 공통 날짜 찾기
        common_dates = sorted(set(price_data.index) & 
                            set(macro_data.index) & 
                            set(sentiment_data.index))
        
        if onchain_data is not None:
            common_dates = sorted(set(common_dates) & set(onchain_data.index))
        
        print(f"공통 날짜 범위: {len(common_dates)}일")
        
        for i in range(self.sequence_length, len(common_dates)):
            end_idx = i - 1
            start_idx = i - self.sequence_length
            target_idx = i
            
            end_date = common_dates[end_idx]
            start_date = common_dates[start_idx]
            target_date = common_dates[target_idx]
            
            # 시퀀스 데이터 추출
            price_seq = price_data.loc[start_date:end_date]
            macro_seq = macro_data.loc[start_date:end_date]
            sentiment_seq = sentiment_data.loc[start_date:end_date]
            
            if len(price_seq) == self.sequence_length and len(macro_seq) == self.sequence_length:
                # 기본 특성 (OHLCV) - 매크로 환경 예측용
                basic_features = price_seq[['open', 'high', 'low', 'close', 'volume']].values.flatten()
                
                # 전체 특성 (OHLCV + 기술적 지표) - 가격 동역학 예측용
                full_features = price_seq.values.flatten()
                
                # 매크로 환경 특성
                macro_features = macro_seq.values.flatten()
                
                # 감정 특성
                if 'sentiment_mean' in sentiment_seq.columns:
                    sentiment_mean = sentiment_seq['sentiment_mean'].values
                    sentiment_std = sentiment_seq['sentiment_std'].values
                    sentiment_count = sentiment_seq['sentiment_count'].values
                    sentiment_features = np.concatenate([sentiment_mean, sentiment_std, 
                                                        np.log1p(sentiment_count)])
                else:
                    sentiment_features = sentiment_seq.values.flatten()
                
                # 온체인 특성 추가
                if onchain_data is not None:
                    try:
                        onchain_seq = onchain_data.loc[start_date:end_date]
                        if len(onchain_seq) == self.sequence_length:
                            onchain_features = onchain_seq.values.flatten()
                            # 온체인 데이터를 full_features에 추가
                            full_features = np.concatenate([full_features, onchain_features])
                    except:
                        pass
                
                # 타겟
                current_price = price_data.loc[end_date, 'close']
                target_price = price_data.loc[target_date, 'close']
                
                sequences.append({
                    'basic_features': basic_features,
                    'full_features': full_features,
                    'macro_features': macro_features,
                    'sentiment_features': sentiment_features,
                    'current_price': current_price,
                    'target_price': target_price,
                    'date': target_date
                })
        
        print(f"생성된 시퀀스: {len(sequences)}개")
        return sequences
    
    def train(self, sequences):
        """
        이중 예측 메커니즘으로 학습
        """
        print(f"\n{'='*70}")
        print("CryptoPulse 모델 학습 시작")
        print(f"{'='*70}")
        
        # 데이터 준비
        macro_X = np.array([seq['basic_features'] for seq in sequences])
        dynamics_X = np.array([seq['full_features'] for seq in sequences])
        sentiment_X = np.array([seq['sentiment_features'] for seq in sequences])
        current_prices = np.array([seq['current_price'] for seq in sequences])
        targets = np.array([seq['target_price'] for seq in sequences])
        
        # 변동량으로 변환 (논문의 핵심)
        target_changes = targets - current_prices
        
        print(f"\n학습 데이터:")
        print(f"  샘플 수: {len(sequences)}")
        print(f"  매크로 특성 차원: {macro_X.shape[1]}")
        print(f"  동역학 특성 차원: {dynamics_X.shape[1]}")
        print(f"  감정 특성 차원: {sentiment_X.shape[1]}")
        print(f"  평균 가격 변동: ${target_changes.mean():.2f} (±${target_changes.std():.2f})")
        
        # 1. 매크로 환경 기반 예측 모델 학습
        print("\n[1/3] 매크로 환경 기반 예측 모델 학습 중...")
        macro_X_scaled = self.scaler_macro.fit_transform(macro_X)
        self.macro_model.fit(macro_X_scaled, target_changes)
        macro_train_pred = self.macro_model.predict(macro_X_scaled)
        macro_train_mae = mean_absolute_error(target_changes, macro_train_pred)
        print(f"      훈련 MAE: ${macro_train_mae:.2f}")
        
        # 2. 가격 동역학 기반 예측 모델 학습
        print("\n[2/3] 가격 동역학 기반 예측 모델 학습 중...")
        dynamics_X_scaled = self.scaler_dynamics.fit_transform(dynamics_X)
        self.dynamics_model.fit(dynamics_X_scaled, target_changes)
        dynamics_train_pred = self.dynamics_model.predict(dynamics_X_scaled)
        dynamics_train_mae = mean_absolute_error(target_changes, dynamics_train_pred)
        print(f"      훈련 MAE: ${dynamics_train_mae:.2f}")
        
        # 3. 융합 모델 학습 (감정 데이터로 두 예측 결합)
        print("\n[3/3] 감정 기반 융합 모델 학습 중...")
        fusion_features = np.column_stack([
            macro_train_pred,
            dynamics_train_pred,
            sentiment_X,
            np.mean(macro_X, axis=1),
            np.std(macro_X, axis=1),
            np.mean(dynamics_X, axis=1),
            np.std(dynamics_X, axis=1)
        ])
        
        fusion_X_scaled = self.scaler_fusion.fit_transform(fusion_features)
        self.fusion_model.fit(fusion_X_scaled, target_changes)
        fusion_train_pred = self.fusion_model.predict(fusion_X_scaled)
        fusion_train_mae = mean_absolute_error(target_changes, fusion_train_pred)
        print(f"      훈련 MAE: ${fusion_train_mae:.2f}")
        
        self.trained = True
        
        print(f"\n{'='*70}")
        print("모델 학습 완료!")
        print(f"{'='*70}")
        
        return self
    
    def predict(self, sequences):
        """
        이중 예측 + 감정 기반 융합으로 예측
        """
        if not self.trained:
            raise ValueError("모델이 학습되지 않았습니다. train() 먼저 실행하세요.")
        
        # 데이터 준비
        macro_X = np.array([seq['basic_features'] for seq in sequences])
        dynamics_X = np.array([seq['full_features'] for seq in sequences])
        sentiment_X = np.array([seq['sentiment_features'] for seq in sequences])
        current_prices = np.array([seq['current_price'] for seq in sequences])
        
        # 스케일링
        macro_X_scaled = self.scaler_macro.transform(macro_X)
        dynamics_X_scaled = self.scaler_dynamics.transform(dynamics_X)
        
        # 개별 예측 (변동량)
        macro_changes = self.macro_model.predict(macro_X_scaled)
        dynamics_changes = self.dynamics_model.predict(dynamics_X_scaled)
        
        # 융합 예측
        fusion_features = np.column_stack([
            macro_changes,
            dynamics_changes,
            sentiment_X,
            np.mean(macro_X, axis=1),
            np.std(macro_X, axis=1),
            np.mean(dynamics_X, axis=1),
            np.std(dynamics_X, axis=1)
        ])
        fusion_X_scaled = self.scaler_fusion.transform(fusion_features)
        final_changes = self.fusion_model.predict(fusion_X_scaled)
        
        # 최종 가격 예측
        predictions = current_prices + final_changes
        macro_predictions = current_prices + macro_changes
        dynamics_predictions = current_prices + dynamics_changes
        
        # 방향성 예측
        direction = np.sign(final_changes)
        
        return {
            'predictions': predictions,
            'macro_predictions': macro_predictions,
            'dynamics_predictions': dynamics_predictions,
            'price_changes': final_changes,
            'direction': direction,  # 1: 상승, -1: 하락, 0: 변화없음
            'dates': [seq['date'] for seq in sequences]
        }

print("✓ CryptoPulse 모델 클래스 구현 완료")


✓ CryptoPulse 모델 클래스 구현 완료


In [11]:

# ==================== 4. 전체 파이프라인 실행 ====================

print("\n" + "="*70)
print("CryptoPulse 전체 파이프라인 실행")
print("="*70)

# Step 1: 기술적 지표 계산
print("\n[STEP 1] 기술적 지표 계산 중...")
model = CryptoPulseModel(sequence_length=7)
eth_with_indicators = model.compute_technical_indicators(eth_price)
print(f"✓ 기술적 지표 추가 완료: {eth_with_indicators.shape}")
print(f"  컬럼: {list(eth_with_indicators.columns)}")

# Step 2: 시퀀스 생성
print("\n[STEP 2] 시퀀스 데이터 생성 중...")
sequences = model.create_sequences(
    price_data=eth_with_indicators,
    macro_data=top5_crypto,
    sentiment_data=daily_sentiment,
    onchain_data=onchain
)

# Step 3: 학습/테스트 분할 (시계열이므로 chronological split)
train_ratio = 0.7
val_ratio = 0.1
test_ratio = 0.2

train_size = int(len(sequences) * train_ratio)
val_size = int(len(sequences) * val_ratio)

train_sequences = sequences[:train_size]
val_sequences = sequences[train_size:train_size+val_size]
test_sequences = sequences[train_size+val_size:]

print(f"\n[STEP 3] 데이터 분할 완료:")
print(f"  학습: {len(train_sequences)}개 ({train_ratio*100:.0f}%)")
print(f"  검증: {len(val_sequences)}개 ({val_ratio*100:.0f}%)")
print(f"  테스트: {len(test_sequences)}개 ({test_ratio*100:.0f}%)")

# Step 4: 모델 학습
print(f"\n[STEP 4] 모델 학습...")
model.train(train_sequences)

# Step 5: 검증 세트 평가
print(f"\n[STEP 5] 검증 세트 평가...")
val_results = model.predict(val_sequences)

val_actual = np.array([seq['target_price'] for seq in val_sequences])
val_pred = val_results['predictions']

val_mae = mean_absolute_error(val_actual, val_pred)
val_mse = mean_squared_error(val_actual, val_pred)
val_rmse = np.sqrt(val_mse)
val_r2 = r2_score(val_actual, val_pred)
val_mape = np.mean(np.abs((val_actual - val_pred) / val_actual)) * 100

print(f"\n검증 세트 성능:")
print(f"  MAE:  ${val_mae:.2f}")
print(f"  RMSE: ${val_rmse:.2f}")
print(f"  MAPE: {val_mape:.2f}%")
print(f"  R²:   {val_r2:.4f}")

# Step 6: 테스트 세트 평가
print(f"\n[STEP 6] 테스트 세트 평가...")
test_results = model.predict(test_sequences)

test_actual = np.array([seq['target_price'] for seq in test_sequences])
test_pred = test_results['predictions']
test_macro_pred = test_results['macro_predictions']
test_dynamics_pred = test_results['dynamics_predictions']

# 성능 지표 계산
test_mae = mean_absolute_error(test_actual, test_pred)
test_mse = mean_squared_error(test_actual, test_pred)
test_rmse = np.sqrt(test_mse)
test_r2 = r2_score(test_actual, test_pred)
test_mape = np.mean(np.abs((test_actual - test_pred) / test_actual)) * 100

# 개별 예측 성능
macro_mae = mean_absolute_error(test_actual, test_macro_pred)
dynamics_mae = mean_absolute_error(test_actual, test_dynamics_pred)

print(f"\n{'='*70}")
print("최종 테스트 세트 성능 (CryptoPulse)")
print(f"{'='*70}")
print(f"\n융합 예측:")
print(f"  MAE:  ${test_mae:.2f}")
print(f"  RMSE: ${test_rmse:.2f}")
print(f"  MAPE: {test_mape:.2f}%")
print(f"  R²:   {test_r2:.4f}")

print(f"\n개별 예측 성능 비교:")
print(f"  매크로 환경 예측 MAE:  ${macro_mae:.2f}")
print(f"  가격 동역학 예측 MAE:  ${dynamics_mae:.2f}")
print(f"  융합 예측 MAE:        ${test_mae:.2f} ⭐")

# 방향성 정확도
actual_direction = np.sign(test_actual - np.array([seq['current_price'] for seq in test_sequences]))
pred_direction = test_results['direction']
direction_accuracy = np.mean(actual_direction == pred_direction) * 100

print(f"\n방향성 예측:")
print(f"  정확도: {direction_accuracy:.2f}%")
print(f"  상승 예측: {np.sum(pred_direction == 1)}일")
print(f"  하락 예측: {np.sum(pred_direction == -1)}일")
print(f"  보합 예측: {np.sum(pred_direction == 0)}일")

print(f"\n{'='*70}")
print("CryptoPulse 파이프라인 완료!")
print(f"{'='*70}")



CryptoPulse 전체 파이프라인 실행

[STEP 1] 기술적 지표 계산 중...
✓ 기술적 지표 추가 완료: (2103, 12)
  컬럼: ['open', 'high', 'low', 'close', 'volume', 'stochastic_k', 'stochastic_d', 'williams_r', 'ad_oscillator', 'momentum', 'disparity_7', 'roc']

[STEP 2] 시퀀스 데이터 생성 중...
공통 날짜 범위: 2059일
생성된 시퀀스: 1825개

[STEP 3] 데이터 분할 완료:
  학습: 1277개 (70%)
  검증: 182개 (10%)
  테스트: 366개 (20%)

[STEP 4] 모델 학습...

CryptoPulse 모델 학습 시작

학습 데이터:
  샘플 수: 1277
  매크로 특성 차원: 35
  동역학 특성 차원: 154
  감정 특성 차원: 21
  평균 가격 변동: $1.98 (±$93.58)

[1/3] 매크로 환경 기반 예측 모델 학습 중...
      훈련 MAE: $36.12

[2/3] 가격 동역학 기반 예측 모델 학습 중...
      훈련 MAE: $55.03

[3/3] 감정 기반 융합 모델 학습 중...
      훈련 MAE: $25.18

모델 학습 완료!

[STEP 5] 검증 세트 평가...

검증 세트 성능:
  MAE:  $98.56
  RMSE: $126.81
  MAPE: 3.17%
  R²:   0.9188

[STEP 6] 테스트 세트 평가...

최종 테스트 세트 성능 (CryptoPulse)

융합 예측:
  MAE:  $124.81
  RMSE: $173.82
  MAPE: 4.00%
  R²:   0.9573

개별 예측 성능 비교:
  매크로 환경 예측 MAE:  $101.20
  가격 동역학 예측 MAE:  $95.17
  융합 예측 MAE:        $124.81 ⭐

방향성 예측:
  정확도: 54.10%
  상승 예측: 178일